In [1]:
%load_ext lab_black
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
from tqdm.auto import tqdm
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset
from torch.utils import data
import warnings

warnings.filterwarnings("ignore")

/srv/scratch/gaoussou/MyEnvVariables/trustiiChallenge/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from pytorch tutorials
transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

transform2 = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

In [3]:
# from medium https://medium.datadriveninvestor.com/covid-19-detection-in-x-ray-images-with-pytorch-5c5602b4658f
train_Aug = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.RandomRotation((-20, 20)),
        torchvision.transforms.RandomAffine(
            0, translate=None, scale=[0.7, 1.3], shear=None, fill=0
        ),
        torchvision.transforms.ToTensor(),
    ]
)
test_Aug = torchvision.transforms.Compose(
    [torchvision.transforms.Resize((224, 224)), torchvision.transforms.ToTensor()]
)

In [4]:
# from https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/

t_train = transforms.Compose(
    [
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

t_val = transforms.Compose(
    [
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

t_test = transforms.Compose(
    [
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

In [5]:
df_train = pd.read_csv("data/train.csv")
df_train.label.unique()

array(['NORMAL', 'PNEUMONIA', 'COVID'], dtype=object)

In [6]:
# custom function to load dataset, we can also use a folder style but this is more straigh i think ;)
class CustomDatasetTrain(Dataset):
    def __init__(self, csv_path, images_folder, transform=None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform
        self.classname = ["NORMAL", "PNEUMONIA", "COVID"]
        self.class2index = {"NORMAL": 0, "PNEUMONIA": 1, "COVID": 2}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        filename = self.df["image_id"].values[index]
        label = self.class2index[self.df["label"].values[index]]
        image = Image.open(os.path.join(self.images_folder, filename)).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [7]:
# custom function to load dataset, we can also use a folder style but this is more straigh i think ;)
class CustomDatasetTest(Dataset):
    def __init__(self, csv_path, images_folder, transform=None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        filename = self.df["image_id"].values[index]
        image = Image.open(os.path.join(self.images_folder, filename))
        if self.transform is not None:
            image = self.transform(image)
        return image, filename

In [8]:
train_dataset = CustomDatasetTrain(
    csv_path="data/train.csv", images_folder="data/data/", transform=train_Aug
)

In [9]:
def toCuda():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    return device

In [10]:
device = toCuda()
device

device(type='cuda', index=0)

# Freezing the weights
for param in model.parameters():
    param.required_grad = False


# Replacing the final layer
model.fc =  nn.Sequential(nn.Linear(512, 256), 
                         nn.ReLU(), 
                         nn.Dropout(p=0.5), 
                         nn.Linear(256, num_classes), 
                         nn.LogSoftmax(dim=1))

model_ft = models.mobilenet_v3_large(pretrained=True)

model_ft

In [11]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            # print(running_loss)
            if phase == "train":
                pass
                # scheduler.step(running_loss)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # deep copy the model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                scheduler.step(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:4f}")

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def train_and_build(model,criterion, optimizer_ft, scheduler, num_epochs=10):
    for epoch in tqdm(range(num_epochs)):
        model.train()
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        running_loss = 0.0
        running_corrects = 0
        for inputs, (images, labels) in enumerate(train_dataset_loader):
            #print(inputs)
            images = images.to(device)
            labels = labels.to(device)
            optimizer_ft.zero_grad()
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_ft.step()
            scheduler.step()
                            # statistics
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        print(f' train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    return model

In [12]:
# from https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/
def trainOnly(epochs, model, optimizer, loss_criterion, train_data_loader):
    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch + 1, epochs))
        # Set to training mode
        model.train()
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0
        for i, (inputs, labels) in enumerate(train_data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Clean existing gradients
            optimizer.zero_grad()
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            # Compute loss
            loss = loss_criterion(outputs, labels)
            # Backpropagate the gradients
            loss.backward()
            # Update the parameters
            optimizer.step()
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            # print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format( i, loss.item(), acc.item()  ))
        avg_train_loss = train_loss / len(train_dataset)
        avg_train_acc = train_acc / float(len(train_dataset))
        exp_lr_scheduler.step(avg_train_loss)
        epoch_end = time.time()
        print(
            "Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%".format(
                epoch,
                avg_train_loss,
                avg_train_acc * 100,
                epoch_end - epoch_start,
            )
        )
    return model

import matplotlib.pyplot as plt


def trainNvalid_v2(model, optimizer, loss_criterion, epochs=25):
    history = []
    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch + 1, epochs))
        # Set to training mode
        model.train()
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0
        for i, (inputs, labels) in enumerate(train_data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Clean existing gradients
            optimizer.zero_grad()
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            # Compute loss
            loss = loss_criterion(outputs, labels)
            # Backpropagate the gradients
            loss.backward()
            # Update the parameters
            optimizer.step()
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            # exp_lr_scheduler.step()
            # print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format( i, loss.item(), acc.item()  ))
        # Validation - No gradient tracking needed
        with torch.no_grad():
            # Set to evaluation mode
            model.eval()
            # Validation loop
            for j, (inputs, labels) in enumerate(val_data_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)
                # Compute loss
                loss = loss_criterion(outputs, labels)
                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)
                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))
                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))
                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)
                # print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
        # Find average training loss and training accuracy
        avg_train_loss = train_loss / len(train)
        avg_train_acc = train_acc / float(len(train))
        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss / len(valid)
        avg_valid_acc = valid_acc / float(len(valid))
        # exp_lr_scheduler.step()
        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
        epoch_end = time.time()
        print(
            "Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, Validation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
                epoch,
                avg_train_loss,
                avg_train_acc * 100,
                avg_valid_loss,
                avg_valid_acc * 100,
                epoch_end - epoch_start,
            )
        )
    plt.plot(history, label="train")
    return model, history

model_ft = train_and_build(model_ft, criterion, optimizer_ft, exp_lr_scheduler)

model_ft = models.vgg16(pretrained=True)
model_ft.classifier[6].in_features

In [14]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
## num_ftrs = model_ft.classifier[2].in_features

# vit_model_b = models.alexnet(pretrained=True)
# num_ftrs = vit_model_b.fc.in_features

# vit_model_l = models.vit_l_16(pretrained=True)
# num_ftrs = vit_model_l.heads.head.in_features
# model_ft = vit_model_l

# model_ft = models.vgg16(pretrained=True)
# model_ft.classifier[6].in_features
# Freezing the weights
# Freeze model parameters
# for param in model_ft.parameters():
#    param.requires_grad = False

# Replacing the final layer
# model_ft.fc =  nn.Sequential(nn.Linear(num_ftrs, 3),   nn.LogSoftmax(dim=1))
# model_ft.head = nn.Sequential(nn.Linear(num_ftrs, 256),nn.ReLU(),nn.Dropout(0.4),nn.Linear(256, 128),nn.ReLU(),nn.Dropout(0.5),nn.Linear(128, 3),
# nn.LogSoftmax(dim=1),  # For using NLLLoss())

# model_ft.fc = torch.nn.Linear(num_ftrs, 3)
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = model_ft.to(device)
# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()
# lr = 3e-5 torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

# Observe that all parameters are being optimized  = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, eps=1e-06)
optimizer_ft = optim.Adam(model_ft.parameters())

exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    verbose=True, mode="max", optimizer=optimizer_ft
)

# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/gaoussou/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 110MB/s] 


In [15]:
train_dataset_loader = data.DataLoader(
    dataset=train_dataset, batch_size=256, shuffle=True
)
train, valid = data.random_split(
    dataset=train_dataset,
    generator=torch.Generator().manual_seed(42),
    lengths=[0.8, 0.2],
)
train_data_loader = data.DataLoader(dataset=train, batch_size=64, shuffle=True)

val_data_loader = data.DataLoader(dataset=valid, batch_size=64, shuffle=True)

In [16]:
dataloaders = {"train": train_data_loader, "val": val_data_loader}
dataset_sizes = {"train": len(train), "val": len(valid)}
class_names = dataloaders["train"].dataset.dataset.classname

model_ft3, history = trainNvalid_v2(  # optimizer, loss_criterion,
    model=model_ft, loss_criterion=criterion, optimizer=optimizer_ft, epochs=50
)

In [17]:
model_ft2 = trainOnly(
    epochs=50,
    model=model_ft,
    optimizer=optimizer_ft,
    loss_criterion=criterion,
    train_data_loader=train_dataset_loader,
)

Epoch: 1/50
Epoch : 000, Training: Loss: 0.3834, Accuracy: 84.5040%
Epoch: 2/50
Epoch : 001, Training: Loss: 0.1605, Accuracy: 94.4247%
Epoch: 3/50
Epoch : 002, Training: Loss: 0.0970, Accuracy: 96.5018%
Epoch: 4/50
Epoch : 003, Training: Loss: 0.0910, Accuracy: 96.6931%
Epoch: 5/50
Epoch : 004, Training: Loss: 0.0753, Accuracy: 97.2670%
Epoch: 6/50
Epoch : 005, Training: Loss: 0.0872, Accuracy: 97.1304%
Epoch: 7/50
Epoch : 006, Training: Loss: 0.0893, Accuracy: 97.1577%
Epoch: 8/50
Epoch : 007, Training: Loss: 0.0524, Accuracy: 98.2782%
Epoch: 9/50
Epoch : 008, Training: Loss: 0.0508, Accuracy: 98.2236%
Epoch: 10/50
Epoch : 009, Training: Loss: 0.0628, Accuracy: 97.6496%
Epoch: 11/50
Epoch : 010, Training: Loss: 0.0674, Accuracy: 97.7316%
Epoch: 12/50
Epoch 00012: reducing learning rate of group 0 to 1.0000e-04.
Epoch : 011, Training: Loss: 0.0622, Accuracy: 97.6770%
Epoch: 13/50
Epoch : 012, Training: Loss: 0.0426, Accuracy: 98.4422%
Epoch: 14/50
Epoch : 013, Training: Loss: 0.0322, 

In [18]:
model_ft3 = train_model(
    model=model_ft2,
    optimizer=optimizer_ft,
    criterion=criterion,
    scheduler=exp_lr_scheduler,
    num_epochs=5,
)

Epoch 0/4
----------
train Loss: 0.0105 Acc: 0.9966
val Loss: 0.0087 Acc: 0.9986

Epoch 1/4
----------
train Loss: 0.0178 Acc: 0.9952
val Loss: 0.0062 Acc: 0.9986

Epoch 2/4
----------
train Loss: 0.0147 Acc: 0.9959
val Loss: 0.0072 Acc: 0.9973

Epoch 3/4
----------
train Loss: 0.0137 Acc: 0.9952
val Loss: 0.0069 Acc: 0.9973

Epoch 4/4
----------
train Loss: 0.0155 Acc: 0.9952
val Loss: 0.0111 Acc: 0.9932

Training complete in 1m 40s
Best val Acc: 0.998632


In [26]:
torch.save(model_ft3.state_dict(), "TrainingOnlyRes50Net2")

In [27]:
from torch.utils import data

test_dataset = CustomDatasetTest(
    csv_path="data/test.csv", images_folder="data/data/", transform=test_Aug
)
test_data_loader = data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)
cat_to_index = {0: "NORMAL", 1: "PNEUMONIA", 2: "COVID"}

In [28]:
model_ft3.eval()
test_data_loader = tqdm(test_data_loader)

pred_list = {}

with torch.no_grad():
    for x, d in enumerate(test_data_loader):
        # print(type(d[1][0]))
        single_im = d[0].to(device)
        im_name = d[1]

        pred = model_ft3(single_im)

        _, index = torch.max(pred, 1)
        # total += label.size(0)
        # correct_preds += (index == label).sum().item()
        # print(index.item(), cat_to_index[index.item()], im_name[0])
        pred_list[im_name[0]] = cat_to_index[index.item()]

df = pd.DataFrame(pred_list.items(), columns=["image_id", "label"])

100%|██████████| 1569/1569 [00:21<00:00, 74.32it/s]


In [29]:
df_test = pd.read_csv("data/test.csv")
# df_test=df_test.set_index('trustii_id')

In [30]:
df_submit = df.merge(df_test, on="image_id")
df_submit = df_submit.set_index("trustii_id")

In [31]:
df_submit

,image_id,label
trustii_id,,
286,7873984.png,PNEUMONIA
708,2220575.png,NORMAL
1295,1208578.png,PNEUMONIA
199,9486255.png,NORMAL
39,6944127.png,NORMAL
...,...,...
432,9628080.png,NORMAL
79,5424375.png,PNEUMONIA
350,1598625.png,NORMAL


In [32]:
df_submit.to_csv("my_submission_gs_test5_resTrainOnlyft50.csv", encoding="UTF-8")